In [1]:
# Dependencies for this notebook can be found in requirements_autosklearn.txt

# Install dependencies using pip3 install -r requirements_autosklearn.txt

# Installation instructions

# virtualenv venv_autosklearn
# source venv_autosklearn/bin/activate
# ipython kernel install --user --name=venv_autosklearn
# Start the notebook using `python3 -m jupyter notebook`
# In the notebook click on Kernel -> Change kernel -> venv_autokeras
# Execute this notebook

# Uncomment these lines if not using a kernal specifically for auto-sklearn


# import sys
# print(sys.executable) 

# ! /usr/local/opt/python@3.9/bin/python3.9 -m pip install auto-sklearn

# You may need to uninstall autogluon using

# # ! /usr/local/opt/python@3.9/bin/python3.9 -m pip uninstall autogluon -y

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/UNSW_NB15_training-set.csv')

In [4]:
df_test = pd.read_csv('data/UNSW_NB15_testing-set.csv')

In [5]:
df = df.drop(['attack_cat', 'id'], axis=1, errors='ignore')

In [6]:
df_test = df_test.drop(['attack_cat', 'id'], axis=1, errors='ignore')

In [26]:
df = df.sample(50000)
df_test = df_test.sample(50000)

In [27]:
num_col = [col for col in df.columns if col not in ['label', 'proto', 'service', 'state']]
num_col

['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']

In [28]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler = scaler.fit(df[num_col])
df[num_col] = scaler.transform(df[num_col])


scaler_test = MinMaxScaler()
scaler_test = scaler_test.fit(df_test[num_col])
df_test[num_col] = scaler.transform(df_test[num_col])

In [29]:
cat_col = ['proto', 'service', 'state']
ohe_df = pd.get_dummies(df[cat_col], drop_first = True)
df = pd.concat([df, ohe_df], axis = 1)

ohe_df_test = pd.get_dummies(df_test[cat_col], drop_first = True)
df_test = pd.concat([df_test, ohe_df_test], axis = 1)

In [30]:
df.drop(cat_col, axis=1, inplace=True, errors='ignore')
df_test.drop(cat_col, axis=1, inplace=True, errors='ignore')

In [32]:
len(df.columns)

187

In [33]:
len(df_test.columns)

189

In [34]:
df_test = df_test[[col for col in df_test.columns if col in df.columns]]

In [35]:
len(df_test.columns)

186

In [36]:
df = df[[col for col in df.columns if col in df_test.columns]]

In [37]:
len(df.columns)

186

In [39]:
df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,state_FIN,state_INT,state_REQ,state_RST
55012,6.666668e-08,0.000094,0.000000,0.000006,0.000000,0.250000,0.996078,0.000000,0.021640,0.000000,...,0,0,0,0,0,0,0,1,0,0
43909,5.862334e-03,0.000845,0.000776,0.000099,0.000026,0.000043,0.996078,0.996047,0.000006,0.000275,...,0,0,0,0,0,0,1,0,0,0
58994,1.666667e-07,0.000094,0.000000,0.000006,0.000000,0.100000,0.996078,0.000000,0.008656,0.000000,...,0,0,0,0,0,0,0,1,0,0
35443,7.562168e-04,0.000470,0.001035,0.000021,0.000175,0.000287,0.121569,0.114625,0.000009,0.015210,...,0,0,0,0,0,0,1,0,0,0
40149,1.663334e-05,0.000094,0.000259,0.000008,0.000017,0.003006,0.121569,0.114625,0.000111,0.038465,...,0,0,0,0,0,0,0,0,0,0


In [40]:
X_train = df[[col for col in df.columns if col != 'label']]
y_train = df['label']

In [41]:
X_test = df_test[[col for col in df_test.columns if col != 'label']]
y_test = df_test['label']

In [42]:
import os

os.environ["OPENBLAS_NUM_THREADS"] = "4"


In [46]:
import autosklearn.classification
cls = autosklearn.classification.AutoSklearnClassifier(
    memory_limit=None,
    time_left_for_this_task=300,
    per_run_time_limit=60,
    tmp_folder="/tmp/autosklearn_classification_example_tmp",
)
cls.fit(X_train, y_train)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      memory_limit=None, per_run_time_limit=60,
                      time_left_for_this_task=300,
                      tmp_folder='/tmp/autosklearn_classification_example_tmp')

In [47]:
print(cls.leaderboard())

          rank  ensemble_weight               type      cost   duration
model_id                                                               
2            1             0.56      random_forest  0.027394  29.273229
10           2             0.14  gradient_boosting  0.031394  14.351459
5            3             0.22           adaboost  0.035818  37.761003
9            4             0.08                lda  0.167152   5.792987


In [48]:
%pprint

Pretty printing has been turned ON


In [50]:
pprint(cls.show_models(), indent=4)

Pretty printing has been turned ON


In [51]:
cls.show_models()

{2: {'model_id': 2,
  'rank': 1,
  'cost': 0.027393939393939415,
  'ensemble_weight': 0.56,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x12ff57640>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x12ff578e0>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x12ff57e80>,
  'sklearn_classifier': RandomForestClassifier(max_features=13, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)},
 5: {'model_id': 5,
  'rank': 2,
  'cost': 0.03581818181818186,
  'ensemble_weight': 0.22,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x12fa091f0>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x130319be0>,
  'cl

In [53]:
y_hat = cls.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

Accuracy score 0.6971
